# STC Jawwy

In [1]:
"""
Here we install libraries that are not installed by default
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb

In [2]:
# Import the required libraries
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of details about each customer and the movies and/or tv shows watched in addition to the genre.

You are required to work on task three to build a recommendation engine for our platform to Recommend movies to usesrs that they might be interested in¶


In [4]:
dataframe = pd.read_excel("/content/stc TV Data Set_T3.xlsx",index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [5]:
# check the data shape
dataframe.shape

(1048575, 5)

In [6]:
# display the first 5 rows
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [7]:
# describe the numeric values in the dataset
dataframe.describe()

,user_id_maped,rating,date_
count,1.048575e+06,1.048575e+06,1048575
mean,1.709266e+04,2.497283e+00,2017-10-04 00:23:20.346183936
min,1.000000e+00,1.000000e+00,2017-03-14 00:00:00
25%,8.253000e+03,1.000000e+00,2017-06-10 00:00:00
50%,1.714900e+04,2.000000e+00,2017-10-14 00:00:00
75%,2.566500e+04,3.000000e+00,2018-01-21 00:00:00
max,3.428000e+04,4.000000e+00,2018-04-30 00:00:00
std,1.003513e+04,1.119837e+00,NaN


In [8]:
# check if any column has null value in the dataset
dataframe.isnull().any()

,0
user_id_maped,False
program_name,False
rating,False
date_,False
program_genre,False


In [9]:
# we import Visualization libraries
# you can ignore and use any other graphing libraries
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
"""
TODO build your Recommender system to Highlight Programs that usesrs might be interested in
"""

# Build simple recommendation system
print("\n" + "="*50)
print("BUILDING RECOMMENDATION SYSTEM")
print("="*50)

def build_recommendation_system(data):
    """Build recommendation system based on similar users"""

    # Create User-Item Matrix
    # Assuming data contains columns: user_id, title, rating or similar

    # Search for appropriate columns in the data
    columns = data.columns.tolist()
    print("Available columns:", columns)

    # Identify user and content columns
    user_col = None
    item_col = None

    for col in columns:
        if 'user' in col.lower() or 'customer' in col.lower():
            user_col = col
        elif 'title' in col.lower() or 'movie' in col.lower() or 'program' in col.lower():
            item_col = col

    print(f"User column: {user_col}")
    print(f"Item column: {item_col}")

    if user_col and item_col:
        # Create interaction matrix (each view = 1)
        user_item_matrix = data.groupby([user_col, item_col]).size().unstack(fill_value=0)
        print(f"User-Item Matrix shape: {user_item_matrix.shape}")

        return user_item_matrix, user_col, item_col
    else:
        print("Could not identify user and item columns")
        return None, None, None

# Build the system
df = dataframe.copy()  # Ensure df exists
user_item_matrix, user_col, item_col = build_recommendation_system(df)

def find_similar_users(user_item_matrix, target_user):
    """Find similar users"""
    if target_user not in user_item_matrix.index:
        return []

    # Calculate similarity between users using simple dot product
    target_user_vector = user_item_matrix.loc[target_user]
    similarities = []

    for other_user in user_item_matrix.index:
        if other_user != target_user:
            other_user_vector = user_item_matrix.loc[other_user]
            # Calculate similarity (number of shared content)
            similarity = np.sum(target_user_vector * other_user_vector)
            if similarity > 0:
                similarities.append((other_user, similarity))

    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities

def get_recommendations_for_user(user_item_matrix, target_user, top_n=5):
    """Get recommendations for a specific user"""
    if target_user not in user_item_matrix.index:
        return []

    # Find similar users
    similar_users = find_similar_users(user_item_matrix, target_user)

    # Content watched by target user
    user_watched = user_item_matrix.loc[target_user]
    watched_items = user_watched[user_watched > 0].index.tolist()

    # Collect recommendations from similar users
    recommendations = {}

    for similar_user, similarity_score in similar_users[:10]:  # Take top 10 similar users
        similar_user_watched = user_item_matrix.loc[similar_user]

        # Content watched by similar user but not by target user
        for item in similar_user_watched[similar_user_watched > 0].index:
            if item not in watched_items:
                if item not in recommendations:
                    recommendations[item] = 0
                recommendations[item] += similarity_score

    # Sort recommendations
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations[:top_n]



BUILDING RECOMMENDATION SYSTEM
Available columns: ['user_id_maped', 'program_name', 'rating', 'date_', 'program_genre']
User column: user_id_maped
Item column: program_genre
User-Item Matrix shape: (11578, 16)


In [14]:
"""
TODO show the recommendations (top 5) for the people who watched "Moana" movie
"""

print("\n" + "="*50)
print("RECOMMENDATIONS FOR MOANA VIEWERS")
print("="*50)

def get_moana_recommendations(data, user_item_matrix, item_col, top_n=5):
    """Get recommendations for people who watched Moana"""

    # Search for Moana movie
    moana_title = None
    if item_col:
        moana_matches = data[data[item_col].str.contains('Moana', case=False, na=False)]
        if not moana_matches.empty:
            moana_title = moana_matches[item_col].iloc[0]
            print(f"Found Moana: {moana_title}")
        else:
            print("Moana not found. Searching for similar titles...")
            # Show sample of available movies
            print("Available titles (sample):")
            sample_titles = data[item_col].unique()[:10]
            for title in sample_titles:
                print(f"- {title}")
            return []

    if moana_title and user_item_matrix is not None:
        # Find users who watched Moana
        if moana_title in user_item_matrix.columns:
            moana_viewers = user_item_matrix[user_item_matrix[moana_title] > 0].index.tolist()
            print(f"Number of Moana viewers: {len(moana_viewers)}")

            # Find other content watched by these users
            other_content = {}

            for viewer in moana_viewers:
                viewer_content = user_item_matrix.loc[viewer]
                watched_content = viewer_content[viewer_content > 0].index.tolist()

                for content in watched_content:
                    if content != moana_title:  # Exclude Moana itself
                        if content not in other_content:
                            other_content[content] = 0
                        other_content[content] += 1

            # Sort content by popularity among Moana viewers
            sorted_content = sorted(other_content.items(), key=lambda x: x[1], reverse=True)

            print(f"\nTop {top_n} recommendations for Moana viewers:")
            print("-" * 60)

            recommendations = []
            for i, (content, viewer_count) in enumerate(sorted_content[:top_n], 1):
                print(f"{i}. {content}")
                print(f"   Watched by {viewer_count} Moana viewers")

                # Add genre information if available
                content_info = data[data[item_col] == content]
                if not content_info.empty and 'genre' in data.columns:
                    genre = content_info['genre'].iloc[0]
                    print(f"   Genre: {genre}")

                recommendations.append((content, viewer_count))
                print()

            return recommendations

        else:
            print(f"Moana ({moana_title}) not found in user-item matrix")
            return []

    return []

# Run the system to get Moana recommendations
moana_recommendations = get_moana_recommendations(df, user_item_matrix, item_col, top_n=5)

# Display additional simple statistics
print("\n" + "="*50)
print("SYSTEM STATISTICS")
print("="*50)

if user_item_matrix is not None:
    print(f"Total users: {user_item_matrix.shape[0]}")
    print(f"Total content items: {user_item_matrix.shape[1]}")
    print(f"Total interactions: {user_item_matrix.sum().sum()}")

    # Most watched content
    most_watched = user_item_matrix.sum().sort_values(ascending=False).head(5)
    print("\nMost watched content:")
    for i, (content, views) in enumerate(most_watched.items(), 1):
        print(f"{i}. {content}: {views} views")

print("\nRecommendation System Complete!")


RECOMMENDATIONS FOR MOANA VIEWERS
Moana not found. Searching for similar titles...
Available titles (sample):
- Drama
- Animation
- Biography
- Action
- Comedy
- Horror
- NOT_DEFINED_IN_UMS
- Thriller
- Documentary
- Crime

SYSTEM STATISTICS
Total users: 11578
Total content items: 16
Total interactions: 1048575

Most watched content:
1. Animation: 401730 views
2. Action: 174155 views
3. Drama: 155048 views
4. Comedy: 136577 views
5. Horror: 75306 views

Recommendation System Complete!
